<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 3 / NOTE 1*

# 📝 Assignment
---

# 🌱 **기본 과제**

In [1]:
!pip install category_encoders 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.1 MB/s 


In [2]:
!pip install eli5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 4.1 MB/s 
     |████████████████████████████████| 133 kB 61.9 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=a52e47902b50c197d90bda23622bf80d7423470a712e06c0a6fa148f576adb11
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import eli5

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder 
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from eli5.sklearn import PermutationImportance

## **Part 1. 데이터 준비**

- 데이터셋을 불러오세요.
  - https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset
- 타겟과 특성을 지정해 주세요. 
  - 타겟은 `Attrition`입니다.  
- 훈련/평가 데이터셋을 분리하세요.
  - 훈련/평가 데이터의 비율은 8:2로 합니다.
  - `random_state=42`로 설정합니다. 


In [4]:
# 데이터셋 불러오기 
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/Employee-Attrition.csv')
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [18]:
target = 'Attrition'
features = [df.drop(columns = 'Attrition').columns]

In [11]:
train,test = train_test_split(df,test_size = 0.2 , random_state=42)

In [ ]:
df["Attrition"] = pd.Categorical(df["Attrition"]).codes

In [15]:
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

## **Part 2. 모델 학습**

- 타겟 Label을 `No=0`, `Yes=1`로 변환하세요.
  - 훈련/평가 데이터 모두 변환합니다. 
- 훈련 데이터셋에 대하여 기준모델의 AUC score를 확인하세요.
  - 기준모델은 최빈값으로 만듭니다.
- `OrdinalEncoder()`를 사용하여 특성을 인코딩하세요. 
- `RandomForestClassifier()`를 사용하여 랜덤포레스트 모델을 학습하세요. 
  - `random_state=42`로 설정합니다. 
  - `n_jobs=-1`, `max_depth=3`으로 설정합니다. 
- `Cross_val_score()`를 활용해서 학습시킨 랜덤포레스트 모델의 검증 score를 구하세요.
  - `cv=10`, `n_jobs=-1`로 설정합니다. 
  - `scoring`은 `roc_auc`를 사용합니다.
  - 검증 score는 평균값으로 구할 수 있습니다.

In [19]:
y_train.head()

1097    0
727     0
254     0
1175    0
1341    0
Name: Attrition, dtype: int8

In [26]:
y_train.value_counts(normalize = True)

0    0.831633
1    0.168367
Name: Attrition, dtype: float64

In [29]:
from sklearn.metrics import roc_curve, roc_auc_score


### 기준모델의 auc 구하기

roc_auc_score(y_train,[0] * len(y_train))

0.5

##### **Q1. 랜덤포레스트 모델의 검증 AUC score를 소수점 둘째 자리까지 제출해 주세요.**

In [32]:
## 여기에서 과제를 수행해주세요.

pipe = make_pipeline(
    
 OrdinalEncoder(),
 RandomForestClassifier(
            random_state=42, n_jobs=-1, max_depth=3
        ),
)

pipe.fit(X_train,y_train)


cv_results = cross_val_score(
    pipe, 
    X_train, 
    y_train, 
    cv=10,
    scoring="roc_auc",
    n_jobs=-1
    )

## 답 : 0.80

In [36]:
round(cv_results.mean(),3)

0.802

## **Part 3. 순열중요도**

- `PermutationImportance()`를 사용해 `permuter`를 정의하세요.
  - 모델은 `randomforestclassifier`로 설정합니다.
  - 평가지표는 `roc_auc`로 설정합니다. 
  - `n_iter=5`로 설정합니다. 
  - `random_state=42`로 설정합니다.
- 평가 데이터셋에 대하여 특성에 Ordinal Encoding을 적용하고, `permuter`로 score를 다시 계산하세요. 
- eli5 라이브러리의 `.show_weights`를 사용해 특성별 score를 확인하세요.  
  - `top=None`으로 설정합니다. 
- score가 평균보다 큰 특성들만 추출하여 랜덤포레스트 모델을 학습하세요.
  - 타겟과 특성(score가 평균보다 큰)을 지정합니다.
  - 훈련/평가 데이터셋을 분리합니다. 
    - 훈련/평가 데이터의 비율은 8:2로 합니다.
    - `random_state=42`로 설정합니다. 
  - 특성은 Ordinal Encoding을 적용합니다. 
  - `RandomForestClassifier()`를 사용하여 모델을 학습합니다.
    - `random_state=42`로 설정합니다. 
    - `n_jobs=-1`, `max_depth=3`으로 설정합니다. 
- `Cross_val_score()`를 활용해서 학습시킨 랜덤포레스트 모델의 검증 score를 구하세요.
  - `cv=10`, `n_jobs=-1`로 설정합니다. 
  - `scoring`은 `roc_auc`를 사용합니다.
  - 검증 score는 평균값으로 구할 수 있습니다.
- 평가 데이터셋에 대하여 모델의 AUC score를 구하세요.

##### **Q2. 순열중요도 계산 결과 score가 가장 높은 특성의 이름을 제출해 주세요.**

##### **Q3. 상위 특성으로 학습한 랜덤포레스트 모델의 검증 AUC score를 소수점 둘째 자리까지 제출해 주세요.**

##### **Q4. 평가 데이터셋에 대하여 모델의 AUC score를 소수점 둘째 자리까지 제출해 주세요.**

In [37]:
## 여기에서 과제를 수행해주세요.

import eli5
from eli5.sklearn import PermutationImportance

# permuter 정의
permuter = PermutationImportance(
    pipe.named_steps["randomforestclassifier"],  # model
    scoring="roc_auc",  # metric
    n_iter=5,  # 다른 random seed를 사용하여 10번 반복
    random_state=42,
)

# 스코어를 다시 계산합니다.
X_test_preprocessed = pipe[0].transform(X_test)
permuter.fit(X_test_preprocessed, y_test)

PermutationImportance(estimator=RandomForestClassifier(max_depth=3, n_jobs=-1,
                                                       random_state=42),
                      random_state=42, scoring='roc_auc')

In [39]:
feature_names = X_test.columns.tolist()


eli5.show_weights(
    permuter,
    top=None,  # top n 지정 가능, None 일 경우 모든 특성
    feature_names=feature_names,  # list 형식으로 넣어야 합니다
)

Weight,Feature
0.0382 ± 0.0328,OverTime
0.0171 ± 0.0181,Age
0.0123 ± 0.0231,StockOptionLevel
0.0087 ± 0.0048,EnvironmentSatisfaction
0.0086 ± 0.0044,DistanceFromHome
0.0082 ± 0.0055,JobInvolvement
0.0077 ± 0.0077,TotalWorkingYears
0.0077 ± 0.0067,JobSatisfaction
0.0067 ± 0.0038,BusinessTravel
0.0057 ± 0.0027,JobRole


## 가장 중요도가 높은 칼럼 : OverTime

In [42]:
permuter.feature_importances_.mean()

0.003208233519652152

# 🥇 **도전 과제**

## **Part 4. 프로젝트 준비**

> 다음주부터 프로젝트가 진행됩니다. <br>
> 관심있는 프로젝트 주제와 데이터를 살펴보시고, 데이터가 정해졌다면 해당 데이터로 도전과제를 진행해보세요.

- 프로젝트에 사용할 데이터셋을 불러오세요.
- 모든 특성에 대하여 모델을 학습하고, 평가지표를 구하세요. 
  - 타겟과 문제에 적합한 모델을 사용합니다. 
- 순열중요도를 계산하세요.
- 순열중요도의 상위 특성만 사용하여 모델을 학습하고, 평가지표를 구하세요. 
- 모든 특성에 대하여 모델을 학습했을 때와 상위 특성만 사용하여 모델을 학습했을 때의 평가지표를 비교해 보세요. 